In [2]:
from abc import ABC, abstractmethod
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass, field

from dbtoolkit.logging_configuration import setup_logging_for_this_script
# setup_logging_for_this_script()
# # Get the logger for this module
# logger = logging.getLogger(__name__)

In [ ]:
def validate_and_convert_datetime(dt: datetime) -> datetime:
    """
    Validates if the input is a datetime object with a UTC timezone. If not, converts it to the correct format.

    Parameters:
    -----------
    dt : datetime
        The input datetime object to validate and convert.

    Returns:
    --------
    datetime
        A timezone-aware datetime object in UTC.
    """
    # Check if the input is already a pandas Timestamp or Python datetime in UTC
    if isinstance(dt, pd.Timestamp):
        if dt.tz is None:
            logger.warning(f"Datetime '{dt}' is not timezone-aware. Converting to UTC.")
            dt = dt.tz_localize("UTC")
        elif dt.tzname() != "UTC":
            logger.warning(f"Datetime '{dt}' is not in UTC. Converting to UTC.")
            dt = dt.tz_convert("UTC")
        else:
            # Correct format and timezone; no need to convert
            logger.info(f"Datetime '{dt}' is already in the correct format.")
            return dt
    elif isinstance(dt, datetime):
        if dt.tzinfo is None or dt.tzinfo.utcoffset(dt) is None:
            logger.warning(f"Datetime '{dt}' is not timezone-aware. Converting to UTC.")
            dt = dt.replace(tzinfo=datetime.timezone.utc)
        elif dt.tzinfo != datetime.timezone.utc:
            logger.warning(f"Datetime '{dt}' is not in UTC. Converting to UTC.")
            dt = dt.astimezone(datetime.timezone.utc)
        else:
            # Correct format and timezone; no need to convert
            logger.info(f"Datetime '{dt}' is already in the correct format.")
            return dt
    else:
        logger.warning(
            f"Input '{dt}' is not a recognized datetime format. Converting to UTC."
        )
        dt = pd.to_datetime(dt, utc=True)

    return dt


def validate_datetime_order(start_datetime: datetime, end_datetime: datetime):
    """
    Validates that the end_datetime is after the start_datetime.

    Parameters:
    -----------
    start_datetime : datetime
        The start datetime of the time series.
    end_datetime : datetime
        The end datetime of the time series.

    Raises:
    -------
    ValueError
        If end_datetime is not after start_datetime.
    """
    if end_datetime <= start_datetime:
        logger.error(
            f"end_datetime ({end_datetime}) must be after start_datetime ({start_datetime})."
        )
        raise ValueError(
            f"end_datetime ({end_datetime}) must be after start_datetime ({start_datetime})."
        )


def validate_time_range(start_datetime: pd.Timestamp, end_datetime: pd.Timestamp, frequency: str):
    """
    Validates that the time range between start_datetime and end_datetime
    is greater than or equal to the provided frequency.

    Parameters:
    -----------
    start_datetime : pd.Timestamp
        The start of the time range.
    end_datetime : pd.Timestamp
        The end of the time range.
    frequency : str
        The frequency at which data should be generated (e.g., '30s', '1min').

    Raises:
    -------
    ValueError
        If the time range is smaller than the provided frequency.
    """
    # Calculate the time difference between start_datetime and end_datetime
    time_difference = end_datetime - start_datetime
    frequency_timedelta = pd.to_timedelta(frequency)

    # Check if the time difference is smaller than the frequency
    if time_difference < frequency_timedelta:
        error_message = (
            f"Time range is too short: {time_difference} is smaller than the frequency {frequency_timedelta}. "
            "Start datetime must be before end datetime by at least the frequency."
        )
        # Log the error
        logger.error(error_message)

        # Raise a ValueError with the same message
        raise ValueError(error_message)


In [ ]:


class RawDataGenerator(ABC):
    """
    Docstring 1
    """
    
    @abstractmethod
    def generate_stable_data(self) -> pd.DataFrame:
        """
        Docstring 2
        """
        pass

class AnomalyInjection(ABC):
    """
    Docstring
    """

    @abstractmethod
    def insert_anomalies(self) -> pd.DataFrame:
        """
        Docsting
        """
        pass


@dataclass
class IndustrialPumpRawData(RawDataGenerator):
    start_datetime: datetime
    end_datetime: datetime
    frequency:str = '30s'
    seed_for_random: int = None

    def generate_stable_data(self) -> pd.DataFrame:
        """
        Generate a stable toy dataset for an industrial pump system with correlated variables.

        Parameters:
        -----------
        start_datetime : datetime
            The start datetime of the time series.
        end_datetime : datetime
            The end datetime of the time series.
        seed_for_random : int, optional
            Seed for random number generation for reproducibility. Defaults to None.

        Returns:
        --------
        pd.DataFrame
            A DataFrame containing the generated time series data with a 'Timestamp' index.
        """
        # Validate and convert datetime inputs
        self.start_datetime = validate_and_convert_datetime(self.start_datetime)
        self.end_datetime = validate_and_convert_datetime(self.end_datetime)

        # Validate that end_datetime is after start_datetime
        validate_datetime_order(self.start_datetime, self.end_datetime)

        # Check that the time range is larger than or equal to the frequency
        validate_time_range(self.start_datetime, self.end_datetime, self.frequency)

        # Set the seed for reproducibility
        if self.seed_for_random is not None:
            np.random.seed(self.seed_for_random)

        # Generate a date range
        date_range = pd.date_range(
            start=self.start_datetime, end=self.end_datetime, freq=self.frequency, tz="UTC"
        )
        number_of_rows = len(date_range)

        # Generate base data with correlations
        temperature = np.random.normal(loc=75, scale=1, size=number_of_rows)
        pressure = (
            3
            - 0.01 * (temperature - 75)
            + np.random.normal(loc=0, scale=0.05, size=number_of_rows)
        )
        flow_rate = (
            300
            + 10 * (3 - pressure)
            + np.random.normal(loc=0, scale=5, size=number_of_rows)
        )
        vibration = 0.1 * np.sqrt(flow_rate * pressure) + np.random.normal(
            loc=0, scale=0.05, size=number_of_rows
        )
        humidity = np.random.normal(loc=40, scale=5, size=number_of_rows)

        # Create a DataFrame
        df = pd.DataFrame(
            {
                "Timestamp": date_range,
                "Temperature_C": temperature,
                "Pressure_MPa": pressure,
                "Vibration_mm_s": vibration,
                "Flow_Rate_l_min": flow_rate,
                "Humidity_%": humidity,
            }
        )

        # Set Timestamp as the index
        df.set_index("Timestamp", inplace=True)

        return df




